In [1]:
import os
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

class_names = ['0', '1', '2', '3', 
               '4', '5', '6', '7', 
               '8', '9', 'A', 'B', 
               'C', 'D', 'E', 'F', 
               'G', 'H', 'K', 'L', 
               'M', 'N', 'O', 'P', 
               'R', 'S', 'T', 'U', 
               'V', 'X', 'Y', 'Z']

datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='training',
    class_mode='sparse'
)

validation_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='validation',
    class_mode='sparse'
)

Found 1729 images belonging to 32 classes.
Found 417 images belonging to 32 classes.


In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (5,5), activation='elu', 
                           padding='same',
                           input_shape=(28, 12, 3)),
    tf.keras.layers.Conv2D(128, (5,5), activation='elu',
                           padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3,3), activation='elu', 
                           padding='same'),
    tf.keras.layers.Conv2D(256, (3,3), activation='elu',
                           padding='same'),
    tf.keras.layers.Conv2D(128, (3,3), activation='elu',
                           padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='elu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='softmax')
])

2022-01-07 18:08:37.168503: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-07 18:08:37.168851: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 12, 128)       9728      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 12, 128)       409728    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 6, 128)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 6, 256)        295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 6, 256)        590080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 6, 128)        295040    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 3, 128)         0

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
class stopCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}) :
        if (logs.get('accuracy') > 0.98) :
            print('\nReached 98% accuracy so stopping training')
            self.model.stop_training = True

In [6]:
callback = stopCallback()
model.fit(train_generator, epochs=1000, callbacks=[callback])

Train for 87 steps
Epoch 1/1000
87/87 [==============================] - 71s 814ms/step - loss: 3.1847 - accuracy: 0.1319
Epoch 2/1000
87/87 [==============================] - 76s 871ms/step - loss: 2.1545 - accuracy: 0.4187
Epoch 3/1000
87/87 [==============================] - 62s 709ms/step - loss: 1.5281 - accuracy: 0.5842
Epoch 4/1000
87/87 [==============================] - 53s 614ms/step - loss: 1.1842 - accuracy: 0.6669
Epoch 5/1000
87/87 [==============================] - 49s 567ms/step - loss: 0.9301 - accuracy: 0.7461
Epoch 6/1000
87/87 [==============================] - 49s 561ms/step - loss: 0.8054 - accuracy: 0.7837
Epoch 7/1000
87/87 [==============================] - 49s 560ms/step - loss: 0.6950 - accuracy: 0.8051
Epoch 8/1000
87/87 [==============================] - 49s 558ms/step - loss: 0.6045 - accuracy: 0.8323
Epoch 9/1000
87/87 [==============================] - 49s 565ms/step - loss: 0.5614 - accuracy: 0.8456
Epoch 10/1000
87/87 [==============================] -

In [7]:
model.evaluate(validation_generator, verbose=1)

21/21 [==============================] - 6s 271ms/step - loss: 0.1942 - accuracy: 0.9472


[0.1941687686959908, 0.9472422]

In [18]:
path1 = "../Testing_set/greenparkingchar"
path2 = "../Testing_set/greenparking2char"

all_path = [path1, path2]

final = "../charLabel/"

for path in all_path:
    for file in os.listdir(path) :
        image_mp= mpimg.imread(path + '/' + file) / 255
        image_mp = np.expand_dims(image_mp, axis=0)
        prediction = model.predict(image_mp)
        if (np.max(prediction) < 0.65) : 
            continue 
        print(path + "/" + file)
        shutil.move(path + '/' + file, final + class_names[np.argmax(prediction)] + '/' + file)

../Testing_set/greenparkingchar/48404_4.jpg
../Testing_set/greenparkingchar/50841_5.jpg
../Testing_set/greenparkingchar/50113_8.jpg
../Testing_set/greenparkingchar/49543_3.jpg
../Testing_set/greenparkingchar/51906_2.jpg
../Testing_set/greenparkingchar/46900_2.jpg
../Testing_set/greenparkingchar/51145_2.jpg
../Testing_set/greenparkingchar/193669_4.jpg
../Testing_set/greenparkingchar/193575_1.jpg
../Testing_set/greenparkingchar/47847_5.jpg
../Testing_set/greenparkingchar/192432_6.jpg
../Testing_set/greenparkingchar/52681_1.jpg
../Testing_set/greenparkingchar/193191_8.jpg
../Testing_set/greenparkingchar/194748_1.jpg
../Testing_set/greenparkingchar/53602_7.jpg
../Testing_set/greenparkingchar/46572_7.jpg
../Testing_set/greenparkingchar/49172_6.jpg
../Testing_set/greenparkingchar/51081_3.jpg
../Testing_set/greenparkingchar/194454_4.jpg
../Testing_set/greenparkingchar/47435_0.jpg
../Testing_set/greenparkingchar/48035_1.jpg
../Testing_set/greenparkingchar/52745_0.jpg
../Testing_set/greenparkin

In [9]:
model.save('../Model')

2022-01-07 18:58:28.656272: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../Model/assets
